In [116]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [117]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')


In [118]:
df.head()

,id,headline,url,publisher,date,stock,headline_cleaned,senti_pos_score,senti_neg_score,senti_score,open,close
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",0.125,0.000,0.125,92.13,90.38
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",0.125,0.000,0.125,90.65,90.49
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",0.250,0.125,0.125,86.23,86.13
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",0.000,0.000,0.000,85.00,84.98
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,"['security.n.01', 'neutral.n.01', 'technology....",0.500,0.250,0.250,85.00,84.98


In [119]:
def stockDevelopement(row):
    stock_differenz = row['open'] - row['close']
    return 1 - row['close']/row['open']
    if quotient < 1:
        return -1
    elif quotient > 1:
        return 1
    else: return 0

In [120]:
df['stockPriceChange'] = df.apply(stockDevelopement, axis=1)

In [121]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPrice_Change"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPrice_Change':'_stockPrice_Change'})

In [122]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [123]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [124]:
df = df.set_index('id')
df.sample(10)

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence
id,,,,,,,
569885,Alphabet shares are trading lower in sympathy ...,GOOGL,"['alphabet.n.01', 'share.n.01', 'trade.v.01', ...",1345.55,1314.760,0.022883,alphabet share trade sympathy overall market fear
255840,8 Healthcare Stocks Moving In Tuesday's Pre-Ma...,CLVS,"['healthcare.n.01', 'stock.n.01', 'tuesday.n.0...",7.39,7.790,-0.054127,healthcare stock tuesday session
430534,Epizyme Shares To Resume Trade At 2:30 p.m. EDT,EPZM,"['share.n.01', 'sketch.n.03', 'trade.n.01']",24.50,20.095,0.179796,share sketch trade
943052,Morgan Stanley Maintains Equal-Weight on Oasis...,OAS,"['morgan.n.01', 'stanley.n.01', 'oasis.n.01', ...",3.17,3.100,0.022082,morgan stanley oasis petroleum raise monetary_...
171924,75 Biggest Movers From Yesterday,BLRX,"['large.a.01', 'mover.n.01', 'yesterday.n.01']",1.99,2.310,-0.160804,large mover yesterday
1331787,"Viad Q4 EPS $(0.01) Beats $(0.13) Estimate, Sa...",VVI,"['beat.n.01', 'estimate.n.01', 'sale.n.01', 'g...",67.10,66.650,0.006706,beat estimate sale girl estimate
206770,From Conagra Q1 Earnings Conference Call: Q2 G...,CAG,"['net_income.n.01', 'conference.n.01', 'call.n...",31.07,31.000,0.002253,net_income conference call gross margin degree...
741821,"UPDATE: CFRA On Kroger Notes 'however, over th...",KR,"['update.n.01', 'note.n.01', 'long.a.01', 'ter...",30.65,33.470,-0.092007,update note long term believe shopper drive sa...
374728,"Citigroup Maintains Buy on Dover, Raises Price...",DOV,"['bargain.n.02', 'dover.n.01', 'raise.n.01', '...",116.03,117.130,-0.009480,bargain dover raise monetary_value target


In [125]:
# splitted_set = train_test_split(df, test_size=0.3)
# train_set_un = splitted_set[0]
# test_set_un = splitted_set[1]
# train_set = train_set_un['_headline_sentence'].tolist()
# test_set = test_set_un['_headline_sentence'].tolist()

In [126]:
df_set = df['_headline_sentence'].tolist()

In [127]:
df.shape

(161478, 7)

In [128]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [129]:
count_wm = count_vectorizer.fit_transform(df_set)
tfidf_wm = tfidf_vectorizer.fit_transform(df_set)


In [130]:
count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

In [131]:
df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=df.index, columns=tfidf_tokenz)

In [ ]:
df_tfidfvect = df_tfidfvect.sort_index(ascending=True)

In [209]:
df_tfidfvect.head()
df_countvect.sample(50)

,131,aare,aaron,abacus,abandon,abattoir,abbot,abbreviated,abdominal,abdominal_aortic_aneurysm,...,zebra,zebu,zero,zeus,zhou,ziegler,zinc,zion,zip_up,zone
id,,,,,,,,,,,,,,,,,,,,,
1262060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1329382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
380098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
873546,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
225292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1313858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
cols_to_delete = []
for column in df_tfidfvect:
    
    # print(df_tfidfvect[df_tfidfvect[column] > 0].shape[0])
    if df_tfidfvect[df_tfidfvect[column] > 0].shape[0] < 20:
        cols_to_delete.append(column)


In [20]:
print(len(cols_to_delete))
print(df_tfidfvect.shape[1])


6119
8713


In [21]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)



In [22]:
df_tfidfvect.head()
# df_tfidfvect.shape

,aare,aaron,ability,able,abstraction,acadia,accelerate,acceleration,accept,accounting,...,yard,year,yesterday,yip,york,yuan,yuma,zebra,zion,zone
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df.head()
# df.shape

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,0.018995,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,0.001765,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.001160,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,0.000235,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,0.000235,security neutral technology raise monetary_val...


In [24]:
# merged_df = pd.concat([df.set_index('id'), df_tfidfvect.set_index('id')], axis=1, join='inner')
mergred = df.merge(df_tfidfvect, on='id')

In [25]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence,aare,aaron,ability,...,yard,year,yesterday,yip,york,yuan,yuma,zebra,zion,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,0.018995,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,0.001765,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.001160,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,0.000235,stock travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,0.000235,security neutral technology raise monetary_val...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [27]:
X = mergred.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', '_stockPriceChange', '_headline_sentence'])
Y = mergred['_stockPriceChange']

In [28]:
model = LinearRegression()
model.fit(X, Y)

LinearRegression()

In [29]:
from sklearn import linear_model

In [30]:
# model = linear_model.SGDRegressor()
# model.fit(X,Y)


In [31]:
def predict_stock_change(row):
    return model.predict([row])



In [32]:
# print(test_df.values.tolist())
predictions = X.apply(predict_stock_change, axis=1)

In [33]:
predictions.head()
predictions = predictions.to_frame()


In [34]:
predictions = predictions.rename(columns={0:'pred_stock_change'})
predictions.head()

,pred_stock_change
id,
0,[-0.011224249376366877]
1,[-0.005030917711925247]
2,[0.0031764661894124807]
3,[-0.002291907767475684]
4,[0.006037819801653162]


In [35]:
merged= mergred.merge(predictions, on="id",  )

In [36]:
final_data = merged[['_headline_sentence', '_stockPriceChange', '_stock', 'pred_stock_change']]

In [37]:
final_data["pred_stock_change"] = final_data['pred_stock_change'].apply(lambda value: value[0])

C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/1355074580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data["pred_stock_change"] = final_data['pred_stock_change'].apply(lambda value: value[0])


In [38]:
final_data.head()

,_headline_sentence,_stockPriceChange,_stock,pred_stock_change
id,,,,
0,stock hit high friday,0.018995,A,-0.011224
1,stock hit high wednesday,0.001765,A,-0.005031
2,large mover friday,0.001160,A,0.003176
3,stock travel friday session,0.000235,A,-0.002292
4,security neutral technology raise monetary_val...,0.000235,A,0.006038


In [39]:
from  sklearn import metrics

In [40]:
final_data['_stockPriceChange'] = final_data['_stockPriceChange'].apply(lambda value: value*100)

C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/88934908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['_stockPriceChange'] = final_data['_stockPriceChange'].apply(lambda value: value*100)


In [41]:
final_data['pred_stock_change'] = final_data['pred_stock_change'].apply(lambda value: value*100)

C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/2544345753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['pred_stock_change'] = final_data['pred_stock_change'].apply(lambda value: value*100)


In [42]:
final_data.to_csv('../data/data_final_tfidf_result.csv')

In [48]:
final_data.sample(15)

,_headline_sentence,_stockPriceChange,_stock,pred_stock_change
id,,,,
40255,daily biotechnology pulsation shopping file ba...,7.874016,AFMD,0.152022
1268932,share several car fabrication car component co...,-1.360544,TTM,-1.350537
313118,capital downgrade real_estate_investment_trust...,0.934579,CTRE,1.162304
96456,bargain armor real_estate_investment_trust rai...,2.464455,ARR,1.092036
715017,professional top heart morning august,-6.034483,JWN,1.027447
1160471,share several technology company trade low mar...,2.628839,SNPS,4.360809
1180423,professional top stock watch ardor,6.299213,SRNE,1.057259
323637,large mover yesterday,2.122186,CWCO,0.615083
285683,canadian pacific interruption grain volume record,0.444920,CP,-2.548662


In [72]:
final_data['_stockPriceChange'] = final_data['_stockPriceChange'].apply(lambda value: round(value, 0))
final_data['pred_stock_change'] = final_data['pred_stock_change'].apply(lambda value: round(value, 0))

C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/498728178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['_stockPriceChange'] = final_data['_stockPriceChange'].apply(lambda value: round(value, 0))
C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/498728178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['pred_stock_change'] = final_data['pred_stock_change'].apply(lambda value: round(value, 0))


In [97]:
final_data.sample(10)

,_headline_sentence,_stockPriceChange,_stock,pred_stock_change
id,,,,
501955,top ascent downgrade november,-1.0,FLL,-0.0
985711,share circuit breaker top stock still session ...,0.0,PCG,-7.0
1347382,top ascent downgrade september,-0.0,WDR,-0.0
1318673,vanda drug_company report federal district cou...,-5.0,VNDA,2.0
662273,daily biotechnology pulsation food_and_drug_ad...,-0.0,IMMU,1.0
648971,large mover friday,-4.0,ICD,0.0
710041,neutral retem network lower_berth monetary_val...,-2.0,JNPR,2.0
981948,share several oil company trade low oil moneta...,22.0,PBF,6.0
1261932,update sell evaluation tesla state position pr...,-8.0,TSLA,-2.0


In [74]:
from sklearn import metrics

In [75]:
metrics.mean_absolute_error(final_data['_stockPriceChange'], final_data['pred_stock_change'])

3.8464001288101164

In [78]:
metrics.mean_absolute_error(final_data['_stockPriceChange'], final_data['pred_stock_change'])

3.8464001288101164

In [98]:
final_data['abs_real'] = final_data['_stockPriceChange'].abs()

C:\Users\kaste\AppData\Local\Temp/ipykernel_23280/548706168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['abs_real'] = final_data['_stockPriceChange'].abs()
